
# A Notebook to TreatGeoSelf to easy access gridded climate time series data (Case study:  the Sauk-Suiattle Watershed )
<img src= "http://www.sauk-suiattle.com/images/Elliott.jpg"
style="float:left;width:150px;padding:20px">   
This data is compiled to digitally observe the Sauk-Suiattle Watershed, powered by HydroShare. <br />
<br />
Use this Jupyter Notebook to: <br /> 
Generate a list of available gridded data points in your area of interest, <br /> 
Download Livneh daily 1/16 degree gridded climate data, <br /> 
Download WRF daily 1/16 degree gridded climate data, <br /> 
Compare daily, monthly, and annual temperature and precipitation data. <br /> 
Visualize modeled streamflow results relative to the forcing data and observed streamflow. <br /> 
<br /> <br /> <br /> <img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:120px;padding:20px">  
#### A Watershed Dynamics Model by the Watershed Dynamics Research Group in the Civil and Environmental Engineering Department at the University of Washington 

## 1.  HydroShare Setup and Preparation

To run this notebook, we must import several libaries. These are listed in order of 1) Python standard libraries, 2) hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading and creation, and 3) the observatory_gridded_hydromet library that is downloaded with this notebook. 

If the python library fiona is not installed, please run the following lines in terminal, and choose 'y' when prompted. <br/>
conda install -c conda-forge fiona <br>
pip install fiona

In [ ]:
!pip install descartes

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from utilities import hydroshare
import ogh

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import fiona
from shapely.geometry import shape, point, MultiPolygon, box

from descartes import PolygonPatch
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable

Establish a secure connection with HydroShare by instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets and prints environment variables for several parameters that will be useful for saving work back to HydroShare. 

In [ ]:
hs=hydroshare.hydroshare()
homedir = ogh.mapContentFolder(str(os.environ["HS_RES_ID"]))
print('Data will be loaded from and save to:'+homedir)

If you are curious about where the data is being downloaded, click on the Jupyter Notebook dashboard icon to return to the File System view.  The homedir directory location printed above is where you can find the data and contents you will download to a HydroShare JupyterHub server.  At the end of this work session, you can migrate this data to the HydroShare iRods server as a Generic Resource. 

## 2. Get list of gridded climate points for the watershed

This example uses a shapefile with the watershed boundary of the Sauk-Suiattle Basin, which is stored in HydroShare at the following url: https://www.hydroshare.org/resource/c532e0578e974201a0bc40a37ef2d284/. 

The data for our processing routines can be retrieved using the getResourceFromHydroShare function by passing in the global identifier from the url above.  In the next cell, we download this resource from HydroShare, and identify that the points in this resource are available for downloading gridded hydrometeorology data, based on the point shapefile at https://www.hydroshare.org/resource/ef2d82bf960144b4bfb1bae6242bcc7f/, which is for the extent of North America and includes the average elevation for each 1/16 degree grid cell.  The file must include columns with station numbers, latitude, longitude, and elevation. The header of these columns must be FID, LAT, LONG_, and ELEV or RASTERVALU, respectively. The station numbers will be used for the remainder of the code to uniquely reference data from each climate station, as well as to identify minimum, maximum, and average elevation of all of the climate stations.  The webserice is currently set to a URL for the smallest geographic overlapping extent - e.g. WRF for Columbia River Basin (to use a limit using data from a FTP service, treatgeoself() would need to be edited in observatory_gridded_hydrometeorology utility). 

In [ ]:
#Your model extent
hs.getResourceFromHydroShare('c532e0578e974201a0bc40a37ef2d284')
shapefile = hs.content['wbdhuc12_17110006_WGS84.shp']

# List of available data
hs.getResourceFromHydroShare('ef2d82bf960144b4bfb1bae6242bcc7f')
NAmer = hs.content['NAmer_dem_list.shp']

# Generate list of stations to download
mappingfile = ogh.treatgeoself(shapefile=shapefile, 
                               NAmer=NAmer,
                               folder_path=os.getcwd(), 
                               outfilename='monkeysonatree.csv',
                               buffer_distance=0.06)
print(mappingfile)

In [ ]:
#ogh.renderWatershed(shapefile, outfilepath='watershed_topo.png')

 ## 3. Download climate data 

### Get Daily Meteorologic Data (1915-2011) from Livneh et al. 2013 

The functions used in this section apply to hydrometeorology data within the Continental United States with daily data 1915-2011. <br/>
View data extent at  Livneh, B. (2017). Gridded climatology locations (1/16th degree): Continental United States extent, HydroShare, http://www.hydroshare.org/resource/14f0a6619c6b45cc90d1f8cabc4129af

Please cite: <br/>
Livneh B., E.A. Rosenberg, C. Lin, B. Nijssen, V. Mishra, K.M. Andreadis, E.P. Maurer, and D.P. Lettenmaier, 2013: A Long-Term Hydrologically Based Dataset of Land Surface Fluxes and States for the Conterminous United States: Update and Extensions, Journal of Climate, 26, 9384–9392.<br/>
<br/>
The getClimateData_DailyMET_livneh2013() function reads in the mapping file table, downloads, and unzips the data files for each of the longitude and latitude points. The folder containing the data is within the directory listed as homedir. 

In [ ]:
#Daily_MET_1915_2011 = ogh.getClimateData_DailyMET_livneh2013(homedir, mappingfile)
Daily_MET_1915_2011 = ogh.getClimateData_DailyMET_livneh2013(homedir, mappingfile, subdir='livneh2013/Daily_MET_1915_2011', catalog_label='livneh2013_MET')

### Get Meteorologic climate data (1950-2013) from Livneh et al. 2015

The functions used in this section apply to hydrometeorology data is within the North America area with daily data 1950-2013. View the data extent from this HydroShare resource: Livneh, B. (2017). Gridded climatology locations (1/16th degree): North American extent, HydroShare,  http://www.hydroshare.org/resource/ef2d82bf960144b4bfb1bae6242bcc7f

Please cite: <br/>
Livneh B., T.J. Bohn, D.S. Pierce, F. Munoz-Ariola, B. Nijssen, R. Vose, D. Cayan, and L.D. Brekke, 2015: A spatially comprehensive, hydrometeorological data set for Mexico, the U.S., and southern Canada 1950-2013, Nature Scientific Data, 5:150042, doi:10.1038/sdata.2015.42.<br/>
<br/>
The getClimateData_DailyMET_livneh2015() function reads in the mapping file table, downloads, and unzips the data files for each of the longitude and latitude points. The folder containing the data is within the directory listed as homedir. 


In [ ]:
Daily_MET_1950_2013 = ogh.getClimateData_DailyMET_livneh2015(homedir, mappingfile)

### Get VIC outputs from Livneh et al., 2013 and Livneh et al., 2015

In [ ]:
Daily_VIC_1915_2011 = ogh.getClimateData_DailyVIC_livneh2013(homedir, mappingfile)

In [ ]:
Daily_VIC_1950_2013 = ogh.getClimateData_DailyVIC_livneh2015(homedir, mappingfile)


### Get the raw and bias corrected Daily Weather Research and Forecasting (WRF 1950-2010 Pacific Northwest) from Salathe et al., 2014
<br/>
Please cite 2014 data using: <br/>
Salathé, EP, AF Hamlet, CF Mass, M Stumbaugh, S-Y Lee, R Steed: 2017. Estimates of 21st Century Flood Risk in the Pacific Northwest Based on Regional Scale Climate Model Simulations.  J. Hydrometeorology. DOI: 10.1175/JHM-D-13-0137.1

This data is also available on HydroShare and can be downloaded using the following line of code (copy into code block):
hs.getResourceFromHydroShare('0db969e4cfb54cb18b4e1a2014a26c82')


In [ ]:
Daily_WRFraw_1950_2010 = ogh.getClimateData_DailyMET_rawWRF(homedir, mappingfile)

In [ ]:
Daily_WRFbc_1950_2010 = ogh.getClimateData_DailyMET_bcWRF(homedir, mappingfile)

In [ ]:
print('This is the list of folders in your user space.')
test = [each for each in os.listdir(homedir) if os.path.isdir(each) and not each.startswith(".")]
print test

In [ ]:
#ogh.renderPointsInShape(shapefile=shapefile, NAmer=NAmer, mappingfile=mappingfile, outfilepath='oghcat_Livneh_Salathe.png')
tmp, nstations = ogh.mappingfileToDF(mappingfile)

In [ ]:
# create list of dataframe
all_daily = ogh.read_in_all_files(map_df=tmp,
                                  dataset='livneh2013_MET',
                                  file_start_date=datetime(1915,1,1), 
                                  file_end_date=datetime(2011,12,31),
                                  subset_start_date=datetime(1950,1,1),
                                  subset_end_date=datetime(2010,12,31))

In [ ]:
sorted(all_daily.keys())

In [ ]:
print(os.getcwd())
os.listdir(os.getcwd())

In [ ]:
# needs to be reconstructed

def generateVarTables (listOfDates, dictOfTables, n_stations):
    # NOTE: each table from dictOfTable must contain:
    # tmin_c, tmax_c, precip_mm, wind_m_s
    
    len_listOfDates=len(listOfDates) # number of dates
    
    # Create arrays of for each variable of interest (Tmin, Tmax, Precip).
    # Rows are dates of analysis and columns are the station number
    temp_min_np=np.empty([len_listOfDates,n_stations])
    temp_max_np=np.empty([len_listOfDates,n_stations])
    precip_np=np.empty([len_listOfDates,n_stations])
    wind_np=np.empty([len_listOfDates,n_stations])
    
    # fill in each array with values from each station
    for i in sorted(dictOfTables.keys()):
        temp_min_np[:,i]=dictOfTables[i].tmin_c.values.astype(float)
        temp_max_np[:,i]=dictOfTables[i].tmax_c.values.astype(float)
        precip_np[:,i]=dictOfTables[i].precip_mm.values.astype(float)
        wind_np[:,i]=dictOfTables[i].wind_m_s.values.astype(float)
        
    # generate each variable dataframe with rows as dates and columns as stations
    temp_min_df=pd.DataFrame(temp_min_np, columns=sorted(dictOfTables.keys()), index=listOfDates)    
    temp_max_df=pd.DataFrame(temp_max_np, columns=sorted(dictOfTables.keys()), index=listOfDates)    
    precip_df=pd.DataFrame(precip_np, columns=sorted(dictOfTables.keys()), index=listOfDates)    
    wind_df=pd.DataFrame(wind_np, columns=sorted(dictOfTables.keys()), index=listOfDates)
    
    # Create average temperature data frame as the average of Tmin and Tmax
    temp_avg_df=pd.DataFrame((temp_min_np+temp_max_np)/2, columns=sorted(dictOfTables.keys()), index=listOfDates)
    
    # generate each variable dataframe with rows as dates and columns as stations
    
    
    return(temp_min_df, temp_max_df, precip_df, wind_df, temp_avg_df)

## 4.  Compare Hydrometeorology 

This section performs computations and generates plots of the Livneh 2013, Livneh 2016, and WRF 2014 temperature and precipitation data in order to compare them with each other and observations. The generated plots are automatically downloaded and saved as .png files in the "plots" folder of the user's home directory and inline in the notebook.

### INPUT: Location Name and watershed drainage area (m2)

In [ ]:
loc_name='Sauk Watershed'
streamflow_watershed_drainage_area=1849242318 # square meters

### INPUT: gridded meteorology from Jupyter Hub folders
Data frames for each set of data are stored in a dictionary. The inputs to gridclim_dict() include the folder location and name of the hydrometeorology data, the file start and end, the analysis start and end, and the elevation band to be included in the analsyis (max and min elevation). <br/>  

#### Create a dictionary of climate variables for the long-term mean (ltm) using the default elevation option of calculating a high, mid, and low elevation average.  The dictionary here is initialized with the Livneh et al., 2013 dataset with a dictionary output 'ltm_3bands', which is used as an input to the second time we run gridclim_dict(), to add the Salathe et al., 2014 data to the same dictionary. 

In [ ]:
ltm_3bands = ogh.gridclim_dict(gridclim_folder='livneh2013_MET',
                               gridclimname='liv2013_met_daily',
                               loc_name=loc_name,
                               mappingfile=mappingfile, 
                               file_start_date=datetime(1915,1,1), 
                               file_end_date=datetime(2011,12,31),
                               subset_start_date=datetime(1950,1,1),
                               subset_end_date=datetime(2010,12,31))

## comparison to WRF data from Salathe et al., 2014

In [ ]:
ltm_3bands = ogh.gridclim_dict(gridclim_folder=Daily_WRFraw_1950_2010,
                               gridclimname='wrf2014_met_daily',
                               loc_name=loc_name,
                               mappingfile=mappingfile,
                               file_start_date=datetime(1950,1,1), 
                               file_end_date=datetime(2010,12,31),
                               subset_start_date=datetime(1950,1,1),
                               subset_end_date=datetime(2010,12,31),  
                               df_dict=ltm_3bands)

# 5. Download gridded hydrology

### Get VIC Hydrology Model data (1950-2013) from Livneh et al. 2016

The functions used in this section apply to hydrometeorology data is within the North America area with daily data 1950-2013. View the data extent from this HydroShare resource: Livneh, B. (2017). Gridded climatology locations (1/16th degree): North American extent, HydroShare,  http://www.hydroshare.org/resource/ef2d82bf960144b4bfb1bae6242bcc7f

Please cite: <br/>
Livneh B., T.J. Bohn, D.S. Pierce, F. Munoz-Ariola, B. Nijssen, R. Vose, D. Cayan, and L.D. Brekke, 2015: A spatially comprehensive, hydrometeorological data set for Mexico, the U.S., and southern Canada 1950-2013, Nature Scientific Data, 5:150042, doi:10.1038/sdata.2015.42.<br/>
<br/>
The getClimateData_DailyVIC_USA_livneh2013() function reads in the mapping file table, downloads, and unzips the data files for each of the longitude and latitude points. Values in Canada use a different folder structure and can be downloaded with the function getClimateData_DailyVIC_CAN_livneh2013() The folder containing the data is within the directory listed as homedir. 


In [ ]:
Daily_VIC_1950_2013 = ogh.getClimateData_DailyVIC_livneh2015(homedir, mappingfile)

In [ ]:
sorted(ltm_3bands.keys())

# 6. Compare gridded model to point observations

### Read in  SNOTEL data - assess available data 
If you want to plot observed snotel point precipitation or temperature with the gridded climate data, set to 'Y' 
Give name of Snotel file and name to be used in figure legends. 
File format: Daily SNOTEL Data Report - Historic - By individual SNOTEL site, standard sensors (https://www.wcc.nrcs.usda.gov/snow/snotel-data.html)

In [ ]:
# Sauk
SNOTEL_file = os.path.join(homedir,'ThunderBasinSNOTEL.txt')
SNOTEL_station_name='Thunder Creek'
SNOTEL_file_use_colsnames = ['Date','Air Temperature Maximum (degF)', 'Air Temperature Minimum (degF)','Air Temperature Average (degF)','Precipitation Increment (in)']
SNOTEL_station_elev=int(4320/3.281) # meters

SNOTEL_obs_daily = ogh.read_daily_snotel(file_name=SNOTEL_file, 
                                         usecols=SNOTEL_file_use_colsnames, 
                                         delimiter=',', 
                                         header=58)

# generate the start and stop date
SNOTEL_obs_start_date=SNOTEL_obs_daily.index[0]
SNOTEL_obs_end_date=SNOTEL_obs_daily.index[-1]

# peek
print(SNOTEL_obs_daily.head(5))

### Read in  COOP station data - assess available data
https://www.ncdc.noaa.gov/

In [ ]:
COOP_file=os.path.join(homedir, 'USC00455678.csv') # Sauk
COOP_station_name='Mt Vernon'
COOP_file_use_colsnames = ['DATE','PRCP','TMAX', 'TMIN','TOBS']
COOP_station_elev=int(4.3) # meters

COOP_obs_daily = ogh.read_daily_coop(file_name=COOP_file,
                                     usecols=COOP_file_use_colsnames,
                                     delimiter=',',
                                     header=0)

# generate the start and stop date
COOP_obs_start_date=COOP_obs_daily.index[0]
COOP_obs_end_date=COOP_obs_daily.index[-1]

# peek
print(COOP_obs_daily.head(5))

## Set up VIC dictionary (as an example)  to compare to available data

In [ ]:
ltm_3bands_vic = ogh.gridhydro_dict(gridclim_folder=Daily_VIC_1950_2013,
                               gridclimname='liv2013_vic_daily',
                               loc_name=loc_name,
                               mappingfile=mappingfile,
                               file_start_date=datetime(1915,1,1), 
                               file_end_date=datetime(2011,12,31),
                               subset_start_date=datetime(1950,1,1),  #matched COOP and Snotel
                               subset_end_date=datetime(2010,12,31)
                               model='vic2013')

## 10. Save the results back into HydroShare
<a name="creation"></a>

Using the `hs_utils` library, the results of the Geoprocessing steps above can be saved back into HydroShare.  First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, *content files*.  In addition, we must define the type of resource that will be created, in this case *genericresource*.  

***Note:*** Make sure you save the notebook at this point, so that all notebook changes will be saved into the new HydroShare resource.

In [ ]:
#execute this cell to list the content of the directory
!ls

Create list of files to save to HydroShare. Verify location and names.

In [ ]:
ThisNotebook='Observatory_Sauk_TreatGeoSelf.ipynb' #check name for consistency
climate2013_tar = 'livneh2013.tar.gz'
climate2015_tar = 'livneh2015.tar.gz'
wrf_tar = 'Salathe2014.tar.gz'
!tar -zcf {climate2013_tar} livneh2013
!tar -zcf {climate2015_tar} livneh2015
!tar -zcf {wrf_tar} Salathe2014

files=[ThisNotebook,
       'monkeysonatree.csv',
       'avg_monthly_precip Sauk Watershed.png',
       'avg_monthly_temp Sauk Watershed.png',
       'observatory_gridded_hydromet.py',
        climate2013_tar,
        climate2015_tar,
        wrf_tar]

In [ ]:
# for each file downloaded onto the server folder, move to a new HydroShare Generic Resource
title = 'Results from testing out the TreatGeoSelf utility'
abstract = 'This the output from the TreatGeoSelf utility integration notebook.
keywords = ['Sauk', 'climate', 'Landlab','hydromet','watershed'] 
rtype = 'genericresource'  

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title,
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)